# 前言

RNN 与 LSTM 对比 CNN 更多用于文本问题。所以再学习的时候要结合nlp进行理解。

NLP的教学见NLP文件夹下内容


nlp中最关键的一个概念就是上下文，即一个词的含义，同义词，反义词都可以通过上下文得到。

为了表现上下文，就出现了很多不同的方法

NLP背景知识

面向深度学习的NLP与面向计数的NLP在基础的词语向量化基础原理存在区别，但基本原理都包括基于上下文的出现概率。

在此只考虑基于one-hot表示以及基于tf-idf表示词语的方法


---



Word2Vec

word2vec可以视为一种无监督学习方法。

本质而言，word2vec也是一个神经网络模型，其结构与之前（面向数值、面向图像）的神经网络模型没有区别，也是分为两个基本步骤


>模型训练： 输入值$\rightarrow$ forward()预测$\rightarrow$ 预测值 $\rightarrow$ 最小化 loss(预测值,实际值) $\rightarrow$ backward()反向传播$\rightarrow$更新权参、偏参$\rightarrow$得到优化后模型

>模型应用：输入值$\rightarrow$ 优化后模型 forward()预测$\rightarrow$预测值

不同之处在于输入的值形式不同，输出的值形式不同，总结对比：
* 数值（多元函数）：
  * 输入: 矩阵 [行实例，列属性]  (不考虑mini-batch情况下 );三阶张量 [mini-batch,行实例，列属性]  
  * 输出：单值（全连接层 out=1）
* 图像：
  * 输入: 4阶张量 [mini-batch,channel,H,W]
  * 输出：多值n分类（softmax层 out=n）
* 文本：
  * 输入: 4阶张量 [mini-batch,channel,H,W]
  * 输出：多值n分类（softmax层 out=n）


  ----

  one-hot可以直接通过sklearn实现，也可以通过自定义包实现。

  word2vec实现包很多，在此通过pytroch实现

  ----------------------------------------------------------------

  

In [2]:
import numpy as np
import pandas as pd
import jieba 
 
 
def token2onehot(words)->pd.DataFrame:
    words_set=sorted(set(words))
    print("分词后list转化为集合，去重，并进行排序处理",words_set)
    diction={}
    for index,value in enumerate(words_set):
       diction[index]=value
    print("转换后的本地词典：",diction)

    column=len(words)
    row=len(diction)
    onehotMatrix=np.zeros((row,column),dtype=float)
    print("one-hot矩阵大小：",onehotMatrix.shape)
    for i in range(len(words)):
        for j in range(len(diction)):
          if words[i]==diction[j]:
              
             onehotMatrix[j,i]=1
    df=pd.DataFrame(onehotMatrix)
    df.columns=words
    return(df)
if __name__=="__main__":
    print("英文one-hot，词典中单词来源于原文")
    sents="you say goodbye and i say hello."
    words=sents.split()
    df=token2onehot(words)
    #print(df)
    #print("中文one-hot，词典中单词来源于原文")
    #sents="中国国家统计局15日公布的70个大中城市房价数据显示"
    #words=list(jieba.cut(sents))
    #df2=token2onehot(words)
    #print(df2)

英文one-hot，词典中单词来源于原文
分词后list转化为集合，去重，并进行排序处理 ['and', 'goodbye', 'hello.', 'i', 'say', 'you']
转换后的本地词典： {0: 'and', 1: 'goodbye', 2: 'hello.', 3: 'i', 4: 'say', 5: 'you'}
one-hot矩阵大小： (6, 7)


1. 使用sklearn进行onenote处理

[onehot features](https://www.youtube.com/watch?v=NxLfpcfGzns&feature=youtu.be)
[onehot sklearn](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/)

2. 基于pyotch实现基本的word2vec对照书中通过numpy实现

   1. 构建模型
   2. 初始化权参
   3. onehot代入模型并进行forward

In [29]:
from sklearn import preprocessing  
from sklearn.preprocessing import OneHotEncoder
import numpy as np 

if __name__=='__main__':
    enc = OneHotEncoder(sparse=False)  # 注意创建的转换器对应稀疏矩阵
    print("英文one-hot，词典中单词来源于原文")
    sents="you say goodbye and i say hello."
    words=sents.split()
    print(type(words))
    words=np.array(words).reshape(-1,1)
    print(words)
    one_hot_word=enc.fit_transform(words)
    
    print(np.array(one_hot_word))
    print(one_hot_word.shape)
    print("onehot字典位置，以及对应的featurename")
    print(enc.get_feature_names())

英文one-hot，词典中单词来源于原文
<class 'list'>
[['you']
 ['say']
 ['goodbye']
 ['and']
 ['i']
 ['say']
 ['hello.']]
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]]
(7, 6)
oneone字典位置
['x0_and' 'x0_goodbye' 'x0_hello.' 'x0_i' 'x0_say' 'x0_you']


c:\Users\tom\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


得到句子中每一个词的onehot表示，可以通过其代入预训练模型，推理出词上下文（词）。

1. 输入一个$n$长度的onehot向量，$n$为句子长度，每个向量单位代表一个词
2. 经过深度学习，一般是全连接层到softmax层
3. 得到同样的$n$长度输出向量，每个向量的值代表对应词是输入词上下文的概率
4. 通过max得到最大概率的词，推理完成



 <img src="figs\word embedding 1.jpg" height="50%" width="50%">

如图所示，word2vec基本的结构与一般神经网络没有太大区别，

假设一句话句子长度为$l$，该句话对应的词典（lexcion）长度为$n$，lexcion去除了句子中重复的词，并可能根据字母排序。

通过one-hot处理，该句话每个词对应一个长度为$l$向量，在词典出现位置上标识1，没有出现位置标识0。

整个句子就可以用$m \times l$矩阵表示。

---

此时目标预测一个词（在句子位置为$w_i$)后面一个词（位置为$w_{i-1}$）

假设此时神经网络已经经过训练，得到了优化的权参，第一层全连接层

1. one-hot 输入向量$A_{1 \times n}$
2. 全连接层中，权参矩阵 $W_{n \times m}$ 
3. 得到中间层结果向量$Z_{ \times m}=A_{1 \times n} \cdot W_{}$
4. 最后输出层softmax层输出为$l$长度向量，每个向量对应lexcion，

In [4]:
# define training data
import torch
corpus = [
    'the quick brown fox',
    'jumped over the lazy dog'
]
tokens = []
for sentence in corpus:
    tokens.extend(sentence.split())
word2idx = {w: i for i, w in enumerate(set(tokens))}
idx2word = {i: w for w, i in word2idx.items()}
data = torch.tensor([word2idx[w] for w in tokens], dtype=torch.long)
print(word2idx)
print(data)

{'over': 0, 'jumped': 1, 'fox': 2, 'the': 3, 'quick': 4, 'dog': 5, 'lazy': 6, 'brown': 7}
tensor([3, 4, 7, 2, 1, 0, 3, 6, 5])
